In [1]:
# coding: utf-8

# ### Notebook to perform Zernike polynominal analysis of SKA1-LOW and LOFAR calibration

# In[1]:

import pylab

import matplotlib.pyplot as plt
plt.switch_backend('pdf')

pylab.rcParams['figure.figsize'] = (8.0, 8.0)
# pylab.rcParams['image.cmap'] = 'rainbow'

from telutil import *
from teliono import *

import subprocess
print("Working from Git repository %s" % subprocess.check_output(["git", "describe", "--long", "--all"]))

def timestamp():
    from datetime import datetime
    print("%s %s" % (datetime.now().date(), datetime.now().time()))


def trials(ntrials, nsources, config, name, rmin=0.3, rmax=200.0, tiono=10, tsky=10000 * 3600.0, snrthreshold=5.0):
    tp = TelArrayPiercing()

    timestamp()

    for trial in range(ntrials):
        ts = TelSources()
        ts.construct(nsources=nsources, radius=wavelength / stationdiameter)
        s = numpy.sqrt(tp.assess(ts, config, nnoll=nnoll, rmax=HWZ, wavelength=3.0, weight=weight, hiono=300,
                                 limit=0.95, doplot=(trial == 0), doFresnel=True))
        print("Trial %d, max(s)=%.1f" % (trial, numpy.max(s)))

        if trial == 0:
            Save = s / float(ntrials)
        else:
            Save = Save + s / float(ntrials)
    J = len(Save[Save > snrthreshold])
    if J < 1:
        J = 1
    DR = TelIono().dr(J, tsky=tsky, B=2 * HWZ)
    itsky = TelIono().tsky(J, DR=1e5, B=2 * HWZ)
    stdphase = numpy.sqrt(TelIono().varphase(J, B=2 * HWZ))

    return {'dr': DR, 's': Save, 'J': J, 'tsky': itsky, 'stdphase': stdphase}


def printstats(stats):
    print("Config, Nsources, peak, J, stdphase, DR, sky, MST")

    for config in stats.keys():
        for nsources in stats[config].keys():
            print("%s, %d, %.1f, %d, %.2f, %.1f, %.1f, %.1f" % (config, nsources, stats[config][nsources]['s'][0],
                                                                stats[config][nsources]['J'],
                                                                stats[config][nsources]['stdphase'],
                                                                10.0 * numpy.log10(stats[config][nsources]['dr']),
                                                                stats[config][nsources]['tsky'] / (
                                                                24.0 * 3600.0 * 365.0),
                                                                mst[config]))

Working from Git repository b'heads/master-0-g26c94d9\n'


In [2]:
random.seed(781490893)

nstations = 512
nnoll = 15000

wavelength = 3.0
stationdiameter = 35.0
rcore = 0.5
rmin = 0.0
hiono = 300.0

HWZ = hiono * wavelength / stationdiameter  # Half width to Zero
FOV = (0.5 * wavelength / stationdiameter) ** 2

rhalo = HWZ
rmax = HWZ
print("FOV (HWZ) at ionosphere=%.1f km" % HWZ)

# Frequency
freq = 3.0e8 / wavelength
print("Observing frequency = %.2f MHz" % (freq / 1e6))

bandwidth = 1e7
print("Observing bandwidth = %.2f MHz" % (bandwidth/1e6))

tiono = 10.0 * numpy.power(wavelength / 3.0, -5.0 / 6.0)
print("Ionospheric coherence time = %.1f (s)" % (tiono))

# Calculate weight of solution for each pierce point
imgnoise = sources().tnoise(time=tiono, freq=freq, bandwidth=bandwidth) * (512 / float(nstations)) * (35.0 /
                                                                                                stationdiameter) ** 2
print("Image noise in %.1f s = %.3f Jy" % (tiono, imgnoise))

visnoise = sources().tnoise(time=tiono, freq=freq, bandwidth=bandwidth) * float(512) * (35.0 / stationdiameter) ** 2
print("Visibility noise in %.1f s = %1.1f Jy" % (tiono, visnoise))

imgthreshold = 10.0 * imgnoise

weight = 1.0 / visnoise ** 2

print("Weight for visibility = %.1f (1/Jy^2)" % (weight))

# Number of sources in the beam abouve 5 sigma in tiono
nsources = int(FOV * sources().numbers(imgthreshold))
print("%.1f sources above image threshold (%.4f Jy/beam)" % (nsources, imgthreshold))

configs = ['LOWBD2', 'LOWBD2-CORE', 'LOWBD2-RASTERHALO', 'LOFAR']

configs = ['LOWBD2']


ntrials = 1

FOV (HWZ) at ionosphere=25.7 km
Observing frequency = 100.00 MHz
Observing bandwidth = 10.00 MHz
Ionospheric coherence time = 10.0 (s)
Image noise in 10.0 s = 0.001 Jy
Visibility noise in 10.0 s = 0.3 Jy
Weight for visibility = 11.0 (1/Jy^2)
1046.0 sources above image threshold (0.0294 Jy/beam)


In [3]:
# ### Define all configs and plot

stats = {}
tel = {}
mst = {}

for config in configs:

    # Define all config variants
    tel[config] = TelArray()
    if config == 'LOWBD2':
        tel[config].readKML(kmlfile='V4Drsst512red_2.kml', weight=weight)
    elif config == 'LOWBD2-CORE':
        tel[config].readCSV('LOWBD2-CORE', l1def='LOWBD2-CORE.csv', recenter=True, weight=weight)
    elif config == 'LOWBD2-HALO':
        tel[config].readCSV('LOWBD2-HALO', l1def='LOWBD2-HALO.csv', recenter=True, weight=weight)
    elif config == 'LOWBD2-RASTERHALO':
        tel['RASTERHALO'] = TelArray()
        tel['RASTERHALO'].rasterBoolardy(nhalo=346, name='LOWRASTERHALO', nstations=346, scale=1.05, rhalo=40.0,
                                         rcore=0.0, weight=weight)
        tel[config] = TelArray().add(tel['RASTERHALO'], tel['LOWBD2-CORE'], name=config)
    elif config == 'LOWBD2-RASTERHALO25KM':
        tel['RASTERHALO25KM'] = TelArray()
        tel['RASTERHALO25KM'].rasterBoolardy(nhalo=346, name='LOWRASTERHALO25KM', nstations=346, scale=40.0 / rhalo,
                                             rhalo=rhalo, rcore=0.0, weight=weight)
        tel['LOWBD2-CORE'] = TelArray()
        tel['LOWBD2-CORE'].readCSV('LOWBD2-CORE', l1def='LOWBD2-CORE.csv', recenter=True, weight=weight)
        tel[config] = TelArray().add(tel['RASTERHALO25KM'], tel['LOWBD2-CORE'], name=config)
    elif config == 'LOFAR':
        tel[config].readLOFAR(weight=weight)
    # Plot and save the figure
    tel[config].plot()
    plt.show()
    plt.savefig('%s.pdf' % config)

for config in configs:
    mst[config] = tel[config].mst()

LOWBD2 has 512


In [4]:
lsources = [nsources]

stats = {}

for nsource in lsources:

    for config in configs:
        print("Processing %s %d sources" % (config, nsource))
        timestamp()
        if nsource == lsources[0]:
            stats[config] = {}
        stats[config][nsource] = trials(ntrials, nsource, tel[config], config)

timestamp()

Processing LOWBD2 1046 sources
2016-07-12 18:56:41.197631
2016-07-12 18:56:41.197656
Source fluxes = [0.66791196987615054, 0.66804256702288267, 0.66924904293868626, 0.67029378944400297, 0.67105681537000939, 0.67318930612965755, 0.67320982404199725, 0.67433219631724362, 0.67534092549174984, 0.67555903902308478, 0.67566449885558699, 0.67568507281799484, 0.67643652337596594, 0.67645745185733208, 0.67699223512379392, 0.67703581378049016, 0.67749675106321461, 0.6776908932073219, 0.67787683973044488, 0.68001967455215884, 0.68174952098674657, 0.68191084621990772, 0.68194409817316448, 0.68222542743528036, 0.68232686312133617, 0.68255796171367777, 0.68365342251535866, 0.68507486070970747, 0.68511124415158586, 0.68681643103206957, 0.68689985171435142, 0.68791226839396502, 0.68972970772012265, 0.68984098782190417, 0.69036082091733908, 0.69130457388304112, 0.69301804951916535, 0.69346555465249471, 0.69463942122991906, 0.69529026200321409, 0.69559782330040232, 0.69582166193777517, 0.696085413245265

/Users/timcornwell/Code/low-calibration/zernikecache.py:31: RuntimeWarning: invalid value encountered in double_scalars
  return N.power(int(-1),  k) * fac(n - k) / (fac(k) * fac((n + m) / 2.0 - k) * fac((n - m) / 2.0 - k))
/Users/timcornwell/Code/low-calibration/zernikecache.py:55: RuntimeWarning: invalid value encountered in true_divide
  sum(pre_fac(k, n, m)                             for k in range((n - m) // 2 + 1)))
/Users/timcornwell/Code/low-calibration/zernikecache.py:54: RuntimeWarning: invalid value encountered in multiply
  return (sum(pre_fac(k, n, m) * N.power(rho, (n - 2 * k)) for k in range((n - m) // 2 + 1)) /
/Users/timcornwell/Code/low-calibration/zernikecache.py:54: RuntimeWarning: invalid value encountered in add
  return (sum(pre_fac(k, n, m) * N.power(rho, (n - 2 * k)) for k in range((n - m) // 2 + 1)) /
/Users/timcornwell/Code/low-calibration/zernikecache.py:55: RuntimeWarning: invalid value encountered in double_scalars
  sum(pre_fac(k, n, m)                  

Calculating station 1 to all other stations
Calculating station 2 to all other stations


KeyboardInterrupt: 

In [ ]:
# ### Now plot results for all the configs


plt.clf()

ymax=0.1
Jmax=0

for nsource in lsources:
    for config in stats.keys():
        y = stats[config][nsource]['s']
        if numpy.max(y) > ymax:
            ymax=numpy.max(y)
        x = numpy.arange(nnoll)
        J=x[y<5.0][0]
        if J>Jmax:
            Jmax=J
        plt.semilogy(x, y, label=config)
plt.axes().axhline(5.0, color='grey', linestyle='dotted', label='5 sigma cutoff')
plt.axes().axvline(Jmax, color='grey', linestyle='dotted', label='J=%d' % (int(round(J))))
plt.xlim([1, nnoll])
plt.ylim([1, ymax])
plt.xlabel('Singular vector index')
plt.ylabel('Singular value')
plt.title('Different configurations: %d source' % nsource)

plt.legend(loc="upper right")
plt.show()

plt.savefig('Allconfigs_sources%d.pdf' % nsource)

plt.clf()


In [ ]:
printstats(stats)

In [ ]:
stats